# В данном проекте будет произведен анализ данных платформы онлайн обучения(E-learning)
    - Будут выявляться курсы с высокой и низкой завершаемостью, определяться самые популярные курсы и курсы с самым большим           оттоком, будет произведен когортный(семестровый) анализ пользователей и RFM-анализ.
  

In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from datetime import date,timedelta

In [3]:
assessments = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-il-seregin/first_project/Промежуточный_проект/assessments.csv')
courses = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-il-seregin/first_project/Промежуточный_проект/courses.csv')
studentAssessment = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-il-seregin/first_project/Промежуточный_проект/studentAssessment.csv')
studentRegistration = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-il-seregin/first_project/Промежуточный_проект/studentRegistration.csv')
   
    # code_module — идентификационный код предмета.

    # code_presentation — семестр (Идентификационный код).

    # id_assessment — тест (Идентификационный номер ассессмента).

    # assessment_type — тип теста. Существуют три типа оценивания: оценка преподавателя (TMA), компьютерная оценка (СМА), экзамен по курсу (Exam).

    # date — информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра. Дата начала семестра имеет номер 0 (ноль).

    # weight — вес теста в % в оценке за курс. Обычно экзамены рассматриваются отдельно и имеют вес 100%; сумма всех остальных оценок составляет 100%.

    # module_presentation_length — продолжительность семестра в днях.
    
    # id_student — идентификационный номер студента.

    # date_submitted — дата сдачи теста студентом, измеряемая как количество дней с начала семестра.

    # is_banked — факт перезачета теста с прошлого семестра (иногда курсы перезачитывают студентам, вернувшимся из академического отпуска).

    # score — оценка учащегося в этом тесте. Диапазон составляет от 0 до 100. Оценка ниже 40 неудачная/неуспешная сдача теста.

    # date_registration — дата регистрации студента. Это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).

    # date_unregistration — дата отмены регистрации студента с предмета. У студентов, окончивших курс, это поле остается пустым.

## 1)Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене)

- РАЗВЕДУЕМ ДАННЫЕ

In [6]:
assessments.head(5)

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [7]:
assessments.id_assessment.nunique()

206

In [8]:
assessments.dtypes

code_module           object
code_presentation     object
id_assessment          int64
assessment_type       object
date                 float64
weight               float64
dtype: object

In [9]:
assessments.query(' code_module == "CCC" and assessment_type == "Exam"')

,code_module,code_presentation,id_assessment,assessment_type,date,weight
62,CCC,2014B,24290,Exam,NaN,100.0
63,CCC,2014B,40087,Exam,NaN,100.0
72,CCC,2014J,24299,Exam,NaN,100.0
73,CCC,2014J,40088,Exam,NaN,100.0


In [10]:
assessments.query('assessment_type == "Exam"').groupby(['code_presentation', 'code_module'], as_index=False) \
            .agg({'assessment_type' : 'count'})

,code_presentation,code_module,assessment_type
0,2013B,BBB,1
1,2013B,DDD,1
2,2013B,FFF,1
3,2013J,AAA,1
4,2013J,BBB,1
5,2013J,DDD,1
6,2013J,EEE,1
7,2013J,FFF,1
8,2013J,GGG,1
9,2014B,BBB,1


In [11]:
courses.isna().sum()

code_module                   0
code_presentation             0
module_presentation_length    0
dtype: int64

In [12]:
courses.dtypes

code_module                   object
code_presentation             object
module_presentation_length     int64
dtype: object

In [13]:
courses.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [14]:
studentAssessment

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0
...,...,...,...,...,...
173907,37443,527538,227,0,60.0
173908,37443,534672,229,0,100.0
173909,37443,546286,215,0,80.0
173910,37443,546724,230,0,100.0


In [15]:
studentAssessment.isna().sum()

id_assessment       0
id_student          0
date_submitted      0
is_banked           0
score             173
dtype: int64

In [16]:
studentAssessment.dtypes

id_assessment       int64
id_student          int64
date_submitted      int64
is_banked           int64
score             float64
dtype: object

In [17]:
studentAssessment.id_assessment.nunique()

188

In [18]:
studentAssessment.id_student.nunique()

23369

In [19]:
studentRegistration.isna().sum()

code_module                0
code_presentation          0
id_student                 0
date_registration         45
date_unregistration    22521
dtype: int64

In [20]:
studentRegistration.dtypes

code_module             object
code_presentation       object
id_student               int64
date_registration      float64
date_unregistration    float64
dtype: object

In [21]:
studentRegistration

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN
...,...,...,...,...,...
32588,GGG,2014J,2640965,-4.0,NaN
32589,GGG,2014J,2645731,-23.0,NaN
32590,GGG,2014J,2648187,-129.0,NaN
32591,GGG,2014J,2679821,-49.0,101.0


In [22]:
studentRegistration.id_student.nunique()

28785

In [23]:
studentRegistration.query('date_unregistration == "NaN"').id_student.nunique()

21190

ДЛЯ РЕШЕНИЯ ЗАДАЧИ ОБЪЕДИНЯЕМ ДАТАСЕТЫ ASSESSMENT, STUDENTASSESSMENT В ОДИН
    - Отсеиваем по типу испытания и его результативности так, чтобы остались успешные сдачи экзаменов
    - Вычисляем сколько экзаменов успешно сдал каждый студент
    - Отсеиваем студентов, сдавших только 1 экзамен

In [24]:
task_1 = assessments.merge(studentAssessment, how='inner', on='id_assessment') \
            .query('assessment_type == "Exam" and score >= 40 ') \
            .groupby('id_student', as_index=False) \
            .agg({'id_assessment' : 'count'}) \
            .rename(columns={'id_assessment' : 'exams_passed'}) \
            .query('exams_passed == 1')

- Вычисляем кол-во таких студентов

In [25]:
task_1.id_student.count()

3802

## 2)Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью.

ДЛЯ РЕШЕНИЯ ЗАДАЧИ ОБЪЕДИНЯЕМ ДАТАСЕТЫ ASSESSMENT, STUDENTASSESSMENT В ОДИН 
 - Отсеиваем по типу испытания так, чтобы остались только сдачи экзаменов 
 - Вычисляем сколько экзаменов было проведено по каждому курсу

In [4]:
task_2 = assessments.merge(studentAssessment, how='inner', on='id_assessment') \
            .query('assessment_type == "Exam"') \
            .groupby(['code_module', 'id_assessment'], as_index=False) \
            .agg({'score' : 'count'}) \
            .rename(columns={'score' : 'examinations'})

- Для вычисления завершаемости добавлям в датасет 2 новые колонки: passed_exams(успешно пройденные испытания) и    completion(завершаемость)

In [6]:
task_2['passed_exams'] = assessments.merge(studentAssessment, how='inner', on='id_assessment') \
            .query('assessment_type == "Exam" and score >= 40') \
            .groupby(['code_module', 'id_assessment'], as_index=False) \
            .agg({'score' : 'count'}).score
task_2['completion'] = (100 * task_2.passed_exams / task_2.examinations).round(2)

- Сортируем полученные результаты по значениям завершаемости 
- Сверху таблицы оказыется айдишник самого сложного экзамена, снизу - самого легкого

In [7]:
task_2.sort_values(by = 'completion')

,code_module,id_assessment,examinations,passed_exams,completion
2,DDD,25340,602,504,83.72
1,CCC,24299,1168,1019,87.24
5,DDD,25368,950,842,88.63
0,CCC,24290,747,664,88.89
3,DDD,25354,968,878,90.70
4,DDD,25361,524,485,92.56


- Теперь вычисляем завершаемость целых курсов

In [8]:
modules_completion = task_2.groupby('code_module', as_index=False) \
                              .agg({'examinations' : 'sum', 'passed_exams' : 'sum'})

In [9]:
modules_completion['completion'] = (100 * modules_completion.passed_exams / modules_completion.examinations).round(2)

In [10]:
modules_completion

,code_module,examinations,passed_exams,completion
0,CCC,1915,1683,87.89
1,DDD,3044,2709,88.99


## 3)По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом).

ДЛЯ РЕШЕНИЯ ЗАДАЧИ ОБЪЕДИНЯЕМ ДАТАСЕТЫ ASSESSMENT, STUDENTASSESSMENT В ОДИН 
    - Отсеиваем по типу испытания и его результативности так, чтобы остались успешные сдачи экзаменов 
    - Вычисляем средний срок сдачи экзаменов по каждому предмету(mean_deadlines)

In [13]:
task_3 = assessments.merge(studentAssessment, how='inner', on='id_assessment') \
            .query('assessment_type == "Exam" and is_banked == 0') \
            .groupby('code_module', as_index=False) \
            .agg({'date_submitted' : 'mean'}) \
            .rename(columns={'date_submitted' : 'mean_deadlines'}) \
            .round(2)

In [14]:
task_3

,code_module,mean_deadlines
0,CCC,239.41
1,DDD,237.90


## 4)Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3).

ДЛЯ РЕШЕНИЯ ЗАДАЧИ ИСПОЛЬЗУЕМ ТОЛЬКО ДАТАСЕТ STUDENTREGISTRATION 
    - Вычисляем сколько студентов рагестрировалось на каждый курс 
    - Смотрим 3 курса с самыми большими показателями

In [16]:
task_4_top3 = studentRegistration.groupby('code_module', as_index=False) \
                    .agg({'id_student' : 'nunique'}) \
                    .rename(columns = {'id_student' : 'students'}) \
                    .sort_values('students', ascending=False) \
                    .head(3)
task_4_top3

,code_module,students
1,BBB,7692
5,FFF,7397
3,DDD,5848


In [17]:
task_4_top3_churn = studentRegistration[~studentRegistration.date_unregistration.isna()] \
                    .groupby('code_module', as_index=False) \
                    .agg({'id_student' : 'nunique'}) \
                    .rename(columns = {'id_student' : 'students'}) \
                    .sort_values('students', ascending=False) \
                    .head(3)
task_4_top3_churn

,code_module,students
1,BBB,2314
5,FFF,2249
3,DDD,2065


## 5)Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов.

In [4]:
def generate_cohorts(df):
    all_attempts = df \
        .query("assessment_type == 'Exam'") \
        .groupby('code_presentation', as_index = False) \
        .agg({'id_student':'count'}) \
        .rename(columns={'id_student':'all_attempts'})
    success_attempts = df \
        .query("score >= 40 and assessment_type == 'Exam'") \
        .groupby('code_presentation', as_index = False) \
        .agg({'id_student':'count', 'date_submitted':'mean'}) \
        .rename(columns={'id_student':'success_attempts', 'date_submitted':'mean_time'})
    cohorts = success_attempts.merge(all_attempts, on='code_presentation')
    cohorts['finalizability'] = round(cohorts.success_attempts / cohorts.all_attempts * 100, 2)
    cohorts = cohorts.drop(columns=['success_attempts', 'all_attempts'])
    return cohorts

Применяем функцию к объединенным датасетам ASSESSMENT, STUDENTASSESSMENT

In [6]:
cohort_table = generate_cohorts(assessments.merge(studentAssessment, how='inner', on='id_assessment'))
cohort_table

,code_presentation,mean_time,finalizability
0,2013B,230.164683,83.72
1,2013J,239.509112,90.70
2,2014B,232.997389,90.40
3,2014J,243.680279,87.87


- Выявляем семестр с самой низкой завершаемостью

In [12]:
cohort_table[cohort_table['finalizability'] == cohort_table['finalizability'].min()]['code_presentation']

0    2013B
Name: code_presentation, dtype: object

- Выявляем семестр с самым долгим строком сдачи кусов

In [21]:
cohort_table[cohort_table['mean_time'] == cohort_table['mean_time'].max()]['code_presentation']

3    2014J
Name: code_presentation, dtype: object

## 6) Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров.

ДЛЯ РЕШЕНИЯ ЗАДАЧИ ОБЪЕДИНЯЕМ ДАТАСЕТЫ ASSESSMENT, STUDENTASSESSMENT В ОДИН 
    - Отсеиваем по типу испытания и его результативности так, чтобы остались только экзамены, по которым записаны результаты       - Вычисляем сколько экзаменов сдавал каждый студент, средний срок сдачи экзамена студентом(R) и его средний бал за экзамен(M)

In [20]:
task_6=studentAssessment.merge(assessments, how='inner', on='id_assessment') \
                        .query('assessment_type == "Exam" and score > 0') \
                        .groupby('id_student', as_index=False) \
                        .agg({'id_assessment' : 'count', 'date_submitted' : 'mean', 'score' : 'mean'})

- Для вычисления завершаемости курсов для каждого студента(F) добавлям в датасет 2 новые колонки: passed_exams(успешно пройденные испытания) и completion(завершаемость)

In [21]:
task_6['passed_exams']=studentAssessment.merge(assessments, how='inner', on='id_assessment') \
                        .query('assessment_type == "Exam" and score >= 40') \
                        .groupby('id_student', as_index=False) \
                        .agg({'id_assessment' : 'count'}) \
                        .id_assessment

In [22]:
task_6['completion']=task_6.passed_exams/task_6.id_assessment

- Формируем датасет для создания кластеров студентов 

In [26]:
task_6.isna().sum()

id_student          0
id_assessment       0
date_submitted      0
score               0
passed_exams      533
completion        533
dtype: int64

In [24]:
task_6=task_6.fillna(0) \
             .drop(columns = {'id_assessment', 'passed_exams'}) \
             .rename(columns = {'date_submitted' : 'R', 'completion' : 'F', 'score' : 'M'})

- Разбиваем на 3 диапазона значения метрик(границы метрик) 
- Получаем:
               -  R(1 = 229.0 - 236.0, 2 = 236.0-243.0, 3= 243.0-285.0)
               -  M(1 = 7.0 - 56.0, 2 = 56.0-76.0, 3= 76.0-100.0) 
               -  F(1 = 0.5 - 1.0, 2 = 1.0-1.0, 3= 1.0-2.0)

In [34]:
quantiles = task_6.quantile(q=[0.33,0.66])
quantiles

,id_student,R,M,F
0.33,524527.64,236.0,56.0,1.0
0.66,607456.24,243.0,76.0,1.0


In [35]:
task_6.max()

id_student    1347447.0
R                 285.0
M                 100.0
F                   2.0
dtype: float64

In [36]:
task_6.min()

id_student    23698.0
R               229.0
M                 7.0
F                 0.5
dtype: float64

- Формируем таблицу с кластерами

In [37]:
quantiles = quantiles.to_dict()

def RScore(x,p,d):
     if x <= d[p][0.33]:
         return 1
     elif x <= d[p][0.66]:
         return 2    
     else:
         return 3

task_6['r_quartile'] = task_6['R'].apply(RScore, args=('R',quantiles,))
task_6['f_quartile'] = task_6['F'].apply(RScore, args=('F',quantiles,))
task_6['m_quartile'] = task_6['M'].apply(RScore, args=('M',quantiles,))
task_6['RFMScore'] = task_6.r_quartile.map(str) + task_6.f_quartile.map(str) + task_6.m_quartile.map(str)

In [38]:
task_6

,id_student,R,M,F,r_quartile,f_quartile,m_quartile,RFMScore
0,23698,243.0,80.0,1.0,2,1,3,213
1,24213,236.0,58.0,1.0,1,1,2,112
2,27116,243.0,96.0,1.0,2,1,3,213
3,28046,237.0,40.0,1.0,2,1,1,211
4,28787,243.0,44.0,1.0,2,1,1,211
...,...,...,...,...,...,...,...,...
4092,1301103,244.0,84.0,1.0,3,1,3,313
4093,1311171,229.5,84.0,0.5,1,1,3,113
4094,1332356,240.0,80.5,0.5,2,1,3,213
4095,1345808,242.0,56.0,1.0,2,1,1,211
